## FashionMNIST

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# Download test data
test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:02<00:00, 10334026.45it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 175528.35it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3323225.64it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6109868.98it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(
    training_data,
    batch_size = batch_size)

test_dataloader = DataLoader(
    test_data,
    batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# get cpu, gpu, mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu")

print(f'Using {device} device')

# Define model
class NeuralNetwork(nn.Module):

  # initiate neuron network model
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  # forward data to the network
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model=NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# Assign loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Return the progress
    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1)*len(X)
      print(f"loss: {loss:>7f} [{current:>5f} / {size:>5d}]")

In [11]:
# Design a test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval() # shift the model to the evaluation mode
  test_loss, correct = 0, 0
  with torch.no_grad(): # disable the gradient descent and to use model as inference model
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() # argmax(1) return index
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [13]:
# set the number of epoch to train model
epochs = 5

# train and validate the model
for t in range(epochs):
  print(f"Epoch {t+1}\n -------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 -------------------------
loss: 1.152060 [64.000000 / 60000]
loss: 1.150747 [6464.000000 / 60000]
loss: 0.964961 [12864.000000 / 60000]
loss: 1.116255 [19264.000000 / 60000]
loss: 0.983274 [25664.000000 / 60000]
loss: 1.010451 [32064.000000 / 60000]
loss: 1.050911 [38464.000000 / 60000]
loss: 0.988688 [44864.000000 / 60000]
loss: 1.038505 [51264.000000 / 60000]
loss: 0.962508 [57664.000000 / 60000]
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.977560 

Epoch 2
 -------------------------
loss: 1.035808 [64.000000 / 60000]
loss: 1.054335 [6464.000000 / 60000]
loss: 0.854363 [12864.000000 / 60000]
loss: 1.026319 [19264.000000 / 60000]
loss: 0.897647 [25664.000000 / 60000]
loss: 0.918201 [32064.000000 / 60000]
loss: 0.976803 [38464.000000 / 60000]
loss: 0.916880 [44864.000000 / 60000]
loss: 0.961759 [51264.000000 / 60000]
loss: 0.897812 [57664.000000 / 60000]
Test Error: 
 Accuracy: 67.1%, Avg loss: 0.907527 

Epoch 3
 -------------------------
loss: 0.951088 [64.000000 / 60000]
loss

In [14]:
# Save models
torch.save(model.state_dict(), "model.pth") # save model internal state dictionary
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


In [15]:
# Loading models
model=NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [25]:
# make prediction on new data
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
